In [4]:
PROMPT = "Design a boost converter that takes in a DC input, and increases the DC input to 5 volts."

In [5]:
########################################
########### OPENAI ENV SETUP ###########
########################################

import together

# DESIGNER_OPENAI_KEY = "" 
together.api_key = "b7e20cb7b1acf926630a492fc67ba01ab30c0b481ef023f426a64ad0f93d5b9b"


In [6]:
#########################################
########### UTILITY FUNCTIONS ###########
#########################################

import requests
from bs4 import BeautifulSoup
import re
import PySpice
import atopile
import subprocess



def web_search(query: str, cutoff=5) -> str:
    """
    Takes in a search query and returns a natural language response from scraping the web. Uses the Qwant search engine, and uses Nous Hermes to summarize the top 5 results.

    Args:
    query (str): The search query
    cutoff (int): The number of search results to summarize
    """
    base_url = "https://api.qwant.com/v3/search/web"
    params = {"q": query, "locale": "en_US", "t": "web"}
    header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(base_url, params=params, headers=header)
    items = response.json()["data"]["result"]["items"]["mainline"]
    for item in items:
        if item["type"] == "web":
            items = item["items"][0:cutoff]

    '''
    Format of ITEM in ITEMS: <- only relevant info
    {'title': title,
    'favicon': favicon,
    'url': url,
    'urlPingSuffix': urlPingSuffix,
    'source': source}
    '''

    text_list = []

    for item in items:
        url = item["url"]
        url_response = requests.get(url, headers=header)
        soup = BeautifulSoup(url_response.text, "html.parser")
        text_list.append(re.sub(r'\n\s+', '\n', re.sub(r' +', ' ', re.sub(r'\n{2,}', '\n', soup.get_text().strip()))))
        print("ok")

    summaries = []

    for text in text_list:
        output = together.Complete.create(
            prompt = f"You are a concise, thorough, articulate genius. Your task is to summarize the following text so it can serve as a guide for another user to answer the query \"{query}\":\n{text}\nSummary:\n",
            model = "Open-Orca/Mistral-7B-OpenOrca", 
            max_tokens = 1024, 
            temperature = 0.2,
            top_k = 60,
            top_p = 0.6,
            repetition_penalty = 1.1,
            stop = ["\n"]
        )
        summaries.append(output['output']['choices'][0]['text'])

    return "The following are a summary of the top 5 search results \n\n" + "\n\n".join(summaries)


def write(code: str):
    """
    Takes in a code string, writes it to the .ato file, and compiles it
    
    Args: 
    - code (str): The code to write to the .ato file
    
    """

    with open("project/elec/src/project.ato", "w") as file:
        file.write(code)
    res = subprocess.run(["ato build"], shell=True, cwd="project", capture_output=True, text=True)
    if res.returncode == 0:
        return res.stdout
    else:
        return res.stderr


def SPICE():
    """
    Runs the PySpice environment
    """
    pass

In [8]:

from pprint import pprint
import re

# load the prompt from assets/prompt.txt
with open("assets/prompt.txt", "r") as file:
    system_prompt = file.read()

system_prompt += PROMPT

system_prompt += "\n---\n"

past_actions = "<reasoning>"

count = 0

while True:
    action = together.Complete.create(
        prompt = system_prompt + past_actions,
        model = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO",
        max_tokens = 512,
        temperature = 0.1,
        top_k = 60,
        top_p = 0.6,
        repetition_penalty = 1.1,
        stop = ["</write>", "</reasoning>", "</websearch>"]
    )

    curr_action = action['output']['choices'][0]['text']
    past_actions += curr_action

    curr_action_type = re.findall(r'<([^<>]+)>', past_actions)[-1]

    performed_action = ""

    if curr_action_type == "reasoning":
        pass
    elif curr_action_type == "websearch":
        print("DOING WEBSEARCH")
        performed_action = web_search(curr_action)
    elif curr_action_type == "write":
        print("DOING WRITE")
        code = re.sub(r'<[^<>]+>', '', curr_action, count=1)
        print("CODE", code)
    elif curr_action_type == "SPICE":
        performed_action =  SPICE(curr_action)


    past_actions += f"</{curr_action_type}>"

    print(curr_action + f"</{curr_action_type}>")

    print("performed_action", performed_action)

    input()




1. Research boost converters and their components
2. Determine the components needed for a boost converter
3. Determine the values of the components needed for a boost converter
4. Write the Atopile code for the boost converter
</reasoning>
performed_action 
DOING WRITE

<write>
import Resistor from "generics/resistors.ato"
import Inductor from "generics/inductors.ato"
import Capacitor from "generics/capacitors.ato"
import Diode from "generics/diodes.ato"
import Transistor_npn from "generics/transistors.ato"
import Opamp from "generics/opamps.ato"

module BoostConverter:
    signal Vin
    signal Vout
    signal Vfb
    signal Vref
    signal Vsense
    signal Vsense_filtered
    signal Vsense_filtered_inverted
    signal Vsense_filtered_inverted_inverted
    signal Vsense_filtered_inverted_inverted_inverted
    signal Vsense_filtered_inverted_inverted_inverted_inverted
    signal Vsense_filtered_inverted_inverted_inverted_inverted_inverted
    signal Vsense_filtered_inverted_inverted

KeyboardInterrupt: 